In [1]:
import os
from time import sleep
from pandas import read_csv
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import unacademy_scraper.ua_consts as ua_consts

ModuleNotFoundError: No module named 'pandas'

In [2]:
chrome_options = Options()
chrome_options.add_argument("--no-sandbox")  # Bypass OS security model
chrome_options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.54 Safari/537.36")
#chrome_options.add_argument("--headless") # uncomment this to use chrome in headless mode
chrome_options.add_argument("window-size=1024,768")  # Open Browser in maximized mode
chrome_options.add_argument("disable-infobars")  # Disabling infobars
chrome_options.add_argument("--disable-extensions")  # Disabling extensions
chrome_options.add_argument("--disable-dev-shm-usage")  # Overcome limited resource problems
chrome_options.add_experimental_option("prefs", {'download.default_directory': os.environ['DOWNLOAD_LOCATION']})

In [4]:
# initiate a headless chrome
driver = webdriver.Chrome(options=chrome_options, executable_path= os.environ['CHROME_DRIVER'])

In [5]:
driver.get(ua_consts.UA_BASE_URL)

In [6]:
def login_unacademy():
    """
    The function will login into unacademy
    """
    login_button = driver.find_element_by_css_selector('.e13239452')
    login_button.click()

    # Locate the input field using the placeholder attribute
    mobile_input_field = driver.find_element_by_css_selector('input[placeholder="Enter your mobile number"]')

    # Take mobile number from user
    mobile_number = input("Enter Mobile Number : ")

    # Insert the mobile number
    mobile_input_field.send_keys(mobile_number)

    # Get OTP
    request_otp_button = driver.find_element_by_css_selector('.ehkw11q4.aquilla-button.button.css-1dik306-SendOtp')
    request_otp_button.click()

    otp = input("Enter OTP : ")

    # Locate all OTP input fields using CSS selector
    otp_fields = driver.find_elements_by_css_selector('.aquilla-otpinput-inputfield.otpinput-inputfield.css-d9udvv')

    # Populate each field with a digit from the OTP
    for i, digit in enumerate(otp):
        otp_fields[i].send_keys(digit)

In [74]:
url_list = read_csv('unacademy_scraper/ua_url_to_be_scraped.csv')['URL'].to_list()

In [75]:
for url in url_list:

    driver.get(url)
    sleep(20)

    # we want to create a new folder for every url
    folder_name = driver.current_url.split('course/')[1].split('/')[0]
    new_download_dir = os.path.join(os.environ['DOWNLOAD_LOCATION'], folder_name)

    # Create the directory if it doesn't exist
    if not os.path.exists(new_download_dir):
        os.makedirs(new_download_dir)

    # Set Chrome to download files to the new directory
    driver.execute_cdp_cmd('Page.setDownloadBehavior', {'behavior': 'allow', 'downloadPath': new_download_dir})

    print(f'Download Started For : {folder_name}')

    # Find all parent elements (I'm assuming they're all wrapped in div with a specific class)
    chapter_elements = driver.find_elements(By.CSS_SELECTOR, 'div.itemContainer')

    # ok now start downloading
    for chapter_element in chapter_elements:
        
        # we will use the title to rename the pdf
        chapter_title = chapter_element.text.split('\n')[0]
        
        # check if we are able to get the SVG
        try:
            # We know that the second image is the pdf one hence the index 1
            pdf_svg = chapter_element.find_elements(By.CSS_SELECTOR, 'svg')[1]
            pdf_svg.click()

            # the element comes at parent level
            with_annotation_pdf = driver.find_element(By.XPATH, ".//p[text()='With annotation']")
            with_annotation_pdf.click()

            print(f'Chapter Downloaded : {chapter_title}')

            sleep(1)
        except:
            print(f'Chapter Download Failed : {chapter_title}')

Download Started For : complete-course-on-instrumentation-and-process-control-802
Chapter Downloaded : Introduction to Process Dynamics
Chapter Downloaded : Introduction to Laplace Transform
Chapter Downloaded : Introduction to Input Changes - I
Chapter Downloaded : Doubt Clearing Session
Chapter Downloaded : Comparison of Input Changes - II
Chapter Downloaded : First Order System
Chapter Downloaded : Step Response
Chapter Downloaded : Doubt Clearing Session
Chapter Downloaded : Discussion on KP and Time Constant
Chapter Downloaded : Response of Other Input Changes
Chapter Download Failed : SEPT
Chapter Downloaded : Doubt Clearing Session
Chapter Downloaded : Tank System
Chapter Downloaded : Mercury Thermometer System
Chapter Downloaded : Numerical Session on First Order System
Chapter Downloaded : Doubt Clearing Session
Chapter Downloaded : Introduction to Second Order System
Chapter Downloaded : Underdamped System - I
Chapter Downloaded : Underdamped System - II
Chapter Downloaded : 